In [1]:
rootFile = '/mnt/disk1/skimmed_Run2/reco/testing/UL2018/MC_mu/ttbar_SemiLeptonic/tree_310_Skim_Skim_Skim.root'

In [2]:
## Calculating A_FB
import ROOT, uproot, numpy as np
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.30/04


In [3]:
# Load the ROOT file and access the tree
with uproot.open(rootFile) as file:
    tree = file["Events"]
    print(tree.keys())

    # Extract the relevant branches
    top_lep_pt = tree["Top_lep_pt"].array()
    top_lep_eta = tree["Top_lep_eta"].array()
    top_lep_phi = tree["Top_lep_phi"].array()
    top_lep_mass = tree["Top_lep_mass"].array()

    top_had_pt = tree["Top_had_pt"].array()
    top_had_eta = tree["Top_had_eta"].array()
    top_had_phi = tree["Top_had_phi"].array()
    top_had_mass = tree["Top_had_mass"].array()

    lep_charge = tree["Muon_charge"].array(library="ak")
    lep_pt = tree["Muon_pt"].array(library="ak")
    lep_eta = tree["Muon_eta"].array(library="ak")
    lep_istight = tree["Muon_tightId"].array(library="ak")
    lep_iso = tree["Muon_pfRelIso04_all"].array(library="ak")

    genPDGId = tree["GenPart_pdgId"].array(library="ak")
    genPt = tree["GenPart_pt"].array(library="ak")
    genEta = tree["GenPart_eta"].array(library="ak")
    genPhi = tree["GenPart_phi"].array(library="ak")

['run', 'luminosityBlock', 'event', 'nGenPart', 'GenPart_eta', 'GenPart_mass', 'GenPart_phi', 'GenPart_pt', 'GenPart_genPartIdxMother', 'GenPart_pdgId', 'GenPart_status', 'GenPart_statusFlags', 'Generator_weight', 'Generator_x1', 'Generator_x2', 'Generator_xpdf1', 'Generator_xpdf2', 'Generator_id1', 'Generator_id2', 'LHEWeight_originalXWGTUP', 'nLHEPdfWeight', 'LHEPdfWeight', 'nLHEScaleWeight', 'LHEScaleWeight', 'nPSWeight', 'PSWeight', 'nJet', 'Jet_area', 'Jet_btagDeepFlavB', 'Jet_eta', 'Jet_mass', 'Jet_phi', 'Jet_pt', 'Jet_rawFactor', 'Jet_jetId', 'Jet_puId', 'L1PreFiringWeight_Dn', 'L1PreFiringWeight_Nom', 'L1PreFiringWeight_Up', 'MET_phi', 'MET_pt', 'nMuon', 'Muon_eta', 'Muon_mass', 'Muon_pfRelIso04_all', 'Muon_phi', 'Muon_pt', 'Muon_charge', 'Muon_isGlobal', 'Muon_isTracker', 'Muon_looseId', 'Muon_mediumId', 'Muon_tightId', 'RawMET_phi', 'RawMET_pt', 'Jet_genJetIdx', 'Jet_hadronFlavour', 'Jet_partonFlavour', 'Flag_HBHENoiseFilter', 'Flag_HBHENoiseIsoFilter', 'Flag_CSCTightHaloFilt

In [4]:
import awkward as ak
# Muon charge = charge of highest pt muon
mask = (
    (lep_pt > 27)
    & (abs(lep_eta) < 2.4)
    & (lep_istight)
    & (lep_iso < 0.06)
)

# Apply mask
selected_muons = ak.mask(lep_pt, mask)  # keep structure consistent

# Get index of highest-pt muon per event
# (ak.argmax returns -1 if no element passes, so handle that)
leading_idx = ak.argmax(selected_muons, axis=1, keepdims=True)

# Select charge of that muon
leading_mu_charge = lep_charge[leading_idx]

# Optionally: flatten to a 1D array (each event → single charge)
leading_mu_charge = ak.fill_none(ak.flatten(leading_mu_charge), 0)

In [5]:
leading_mu_charge 

<Array [1, -1, -1, 1, 1, ... 1, 1, 1, -1, 1] type='39002 * int64'>

In [6]:
for i in range(10):
    print(f"Event {i}:,{leading_mu_charge[i],  lep_pt[i], lep_eta[i], lep_istight[i], lep_iso[i], lep_charge[i]}")


Event 0:,(1, <Array [54.4] type='1 * float32'>, <Array [1.06] type='1 * float32'>, <Array [True] type='1 * bool'>, <Array [0] type='1 * float32'>, <Array [1] type='1 * int32'>)
Event 1:,(-1, <Array [68.9, 7.1, 4.42] type='3 * float32'>, <Array [-1.16, 0.613, 0.389] type='3 * float32'>, <Array [True, True, False] type='3 * bool'>, <Array [0, 11.5, 27.1] type='3 * float32'>, <Array [-1, 1, -1] type='3 * int32'>)
Event 2:,(-1, <Array [44.3] type='1 * float32'>, <Array [-0.204] type='1 * float32'>, <Array [True] type='1 * bool'>, <Array [0] type='1 * float32'>, <Array [-1] type='1 * int32'>)
Event 3:,(1, <Array [32] type='1 * float32'>, <Array [1.13] type='1 * float32'>, <Array [True] type='1 * bool'>, <Array [0] type='1 * float32'>, <Array [1] type='1 * int32'>)
Event 4:,(1, <Array [52.8] type='1 * float32'>, <Array [-0.141] type='1 * float32'>, <Array [True] type='1 * bool'>, <Array [0.00452] type='1 * float32'>, <Array [1] type='1 * int32'>)
Event 5:,(-1, <Array [83.4] type='1 * float32

In [7]:
# --- Determine top / anti-top assignment ---
is_positive = leading_mu_charge > 0

Top_pt   = ak.where(is_positive, top_lep_pt,   top_had_pt)
Top_eta  = ak.where(is_positive, top_lep_eta,  top_had_eta)
Top_phi  = ak.where(is_positive, top_lep_phi,  top_had_phi)
Top_mass = ak.where(is_positive, top_lep_mass, top_had_mass)

antiTop_pt   = ak.where(is_positive, top_had_pt,   top_lep_pt)
antiTop_eta  = ak.where(is_positive, top_had_eta,  top_lep_eta)
antiTop_phi  = ak.where(is_positive, top_had_phi,  top_lep_phi)
antiTop_mass = ak.where(is_positive, top_had_mass, top_lep_mass)

print(Top_pt, antiTop_pt)

[83.3, 101, 166, 73.7, 58.7, 49.2, 75.3, ... 18.7, 36.8, 67.3, 66, 131, 192, 71.2] [54.3, 182, 104, 17.5, 68.3, 120, 122, ... 67.2, 91.4, 31.2, 152, 31.3, 288, 49.7]


In [8]:
def to_cartesian(pt, eta, phi, mass):
    px = pt * np.cos(phi)
    py = pt * np.sin(phi)
    pz = pt * np.sinh(eta)
    E  = np.sqrt(px**2 + py**2 + pz**2 + mass**2)
    return px, py, pz, E

# Top and antiTop 4-vectors
Top_px, Top_py, Top_pz, Top_E = to_cartesian(Top_pt, Top_eta, Top_phi, Top_mass)
antiTop_px, antiTop_py, antiTop_pz, antiTop_E = to_cartesian(antiTop_pt, antiTop_eta, antiTop_phi, antiTop_mass)

# ---- Combine and compute invariant mass ----
ttbar_E  = Top_E + antiTop_E
ttbar_px = Top_px + antiTop_px
ttbar_py = Top_py + antiTop_py
ttbar_pz = Top_pz + antiTop_pz

ttbar_mass = np.sqrt(np.maximum(ttbar_E**2 - ttbar_px**2 - ttbar_py**2 - ttbar_pz**2, 0))


In [25]:
ttbar_pz

<Array [56.3, -304, -333, ... -1.75e+03, -47.1] type='39002 * float32'>

In [9]:
Cos_ = Top_pz * ttbar_pz

In [10]:
Nplus = np.sum(Cos_ > 0)
Nminus = np.sum(Cos_ < 0)

In [11]:
aFB = (Nplus - Nminus) / (Nplus + Nminus)

In [12]:
aFB 

0.7244756679144659

In [13]:
# Compute the sum of the first two pdgIds per event
sum_first_two = ak.sum(genPDGId[:, :2], axis=1)

# qqbar if sum == 0 (e.g., 2 + (-2), 4 + (-4), etc.)
qqbar_mask = (sum_first_two == 0)
gg_mask = (sum_first_two == 42)
qg_mask = (sum_first_two > 14) & (sum_first_two < 29)

print(qqbar_mask)

[False, False, False, False, False, False, ... False, False, False, False, False]


In [14]:
selEvents_q1_id = genPDGId[qqbar_mask][:, 0]

In [15]:
q_dir = ak.where(selEvents_q1_id > 0, 1, -1)

In [16]:
sel_ttbar_pz = ttbar_pz[qqbar_mask]

In [17]:
assign = q_dir * sel_ttbar_pz

In [18]:
sum(assign > 0)/len(assign)

0.6875567665758402

In [19]:
cos_theta = Cos_[qqbar_mask]

In [20]:
Nplus_ = np.sum(cos_theta > 0)
Nminus_ = np.sum(cos_theta < 0)

aFB_ = (Nplus_ - Nminus_) / (Nplus_ + Nminus_)

In [21]:
aFB_

0.8038147138964578